In [1]:
import pandas as pd
customers=pd.read_excel("customers.xlsx")
transactions=pd.read_excel("transactions.xlsx")

In [2]:
customers.dtypes
transactions.dtypes

transaction_id            object
customer_id               object
order_date        datetime64[ns]
order_value                int64
items_count                int64
category                  object
dtype: object

In [3]:
snapshot_date=pd.to_datetime("2024-12-31")
tx_before=transactions[transactions["order_date"]<=snapshot_date]
tx_before.shape

(200, 6)

In [4]:
df=tx_before.merge(customers, on="customer_id", how="left")
print(df.shape)
df.head()

(200, 11)


,transaction_id,customer_id,order_date,order_value,items_count,category,signup_date,region,acquisition_channel,gender,age
0,T00001,C0001,2023-02-15,450,2,Electronics,2020-03-12,Delhi,Organic,F,28.0
1,T00002,C0001,2023-05-11,1200,1,Home,2020-03-12,Delhi,Organic,F,28.0
2,T00003,C0002,2024-03-29,980,3,Fashion,2021-07-25,Mumbai,Facebook,M,34.0
3,T00004,C0002,2022-12-28,300,1,Books,2021-07-25,Mumbai,Facebook,M,34.0
4,T00005,C0002,2023-04-16,750,2,Home,2021-07-25,Mumbai,Facebook,M,34.0


In [5]:
import numpy as np
agg=df.groupby("customer_id").agg(frequency=("transaction_id","count"),monetary_sum=("order_value","sum"), monetory_avg=("order_value","mean"),
last_order_date=("order_date","max"), first_order_date=("order_date","min"),distinct_categories=("category","nunique"),avg_items=("items_count","mean")).reset_index()

agg["recencey_days"]=(snapshot_date-agg["last_order_date"]).dt.days
agg["tenure_days"]=(snapshot_date-agg["first_order_date"]).dt.days

In [12]:
print("snapshot_date:",snapshot_date)
print("df shape:", df.shape)
print("sample columns:", df.columns.tolist())

def orders_in_window(df_input, days):
    """Count orders per customer in last days from snapshot_date. returns a series indexed by customer_id with orders."""
    
    cutoff = snapshot_date - pd.Timedelta(days=days)
    tmp = df_input[df_input["order_date"]>cutoff]
    return tmp.groupby("customer_id").size().rename(f"orders_{days}d")

for d in (30,90,180):
    series=orders_in_window(df,d)
    agg=agg.merge(series, on="customer_id",how="left")
    
agg.fillna(0, inplace=True)

print("agg columns:", agg.columns.tolist())
print("agg.head:")
display(agg.head())

snapshot_date: 2024-12-31 00:00:00
df shape: (200, 11)
sample columns: ['transaction_id', 'customer_id', 'order_date', 'order_value', 'items_count', 'category', 'signup_date', 'region', 'acquisition_channel', 'gender', 'age']
agg columns: ['customer_id', 'frequency', 'monetary_sum', 'monetory_avg', 'last_order_date', 'first_order_date', 'distinct_categories', 'avg_items', 'recencey_days', 'tenure_days', 'orders_30d', 'orders_90d', 'orders_180d']
agg.head:


,customer_id,frequency,monetary_sum,monetory_avg,last_order_date,first_order_date,distinct_categories,avg_items,recencey_days,tenure_days,orders_30d,orders_90d,orders_180d
0,C0001,2,1650,825.000000,2023-05-11,2023-02-15,2,1.50,600,685,0.0,0.0,0.0
1,C0002,4,2480,620.000000,2024-03-29,2022-12-28,4,1.75,277,734,0.0,0.0,0.0
2,C0003,4,2790,697.500000,2024-07-19,2023-03-22,4,1.25,165,650,0.0,0.0,1.0
3,C0004,4,4540,1135.000000,2024-11-01,2020-08-12,4,1.50,60,1602,0.0,2.0,2.0
4,C0005,3,1775,591.666667,2024-06-05,2021-06-30,3,1.00,209,1280,0.0,0.0,0.0


In [9]:
print(df.columns.tolist())
print(df.dtypes)
print(agg.columns.tolist())

['transaction_id', 'customer_id', 'order_date', 'order_value', 'items_count', 'category', 'signup_date', 'region', 'acquisition_channel', 'gender', 'age']
transaction_id                 object
customer_id                    object
order_date             datetime64[ns]
order_value                     int64
items_count                     int64
category                       object
signup_date            datetime64[ns]
region                         object
acquisition_channel            object
gender                         object
age                           float64
dtype: object
['customer_id', 'frequency', 'monetary_sum', 'monetory_avg', 'last_order_date', 'first_order_date', 'distinct_categories', 'avg_items', 'recencey_days', 'tenure_days']


In [14]:
agg.to_csv("ltv_features_raw.csv", index=False)
print(agg.shape)
agg.head()

(51, 13)


,customer_id,frequency,monetary_sum,monetory_avg,last_order_date,first_order_date,distinct_categories,avg_items,recencey_days,tenure_days,orders_30d,orders_90d,orders_180d
0,C0001,2,1650,825.000000,2023-05-11,2023-02-15,2,1.50,600,685,0.0,0.0,0.0
1,C0002,4,2480,620.000000,2024-03-29,2022-12-28,4,1.75,277,734,0.0,0.0,0.0
2,C0003,4,2790,697.500000,2024-07-19,2023-03-22,4,1.25,165,650,0.0,0.0,1.0
3,C0004,4,4540,1135.000000,2024-11-01,2020-08-12,4,1.50,60,1602,0.0,2.0,2.0
4,C0005,3,1775,591.666667,2024-06-05,2021-06-30,3,1.00,209,1280,0.0,0.0,0.0
